# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr_model').getOrCreate()

In [65]:
df = spark.read.csv("cruise_ship_info.csv", header=True, inferSchema=True)

In [66]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



- crew is the target
- ship_name and Cruise_line are string and must convert

In [67]:
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler

In [68]:
# convert string to index
for col in ["Ship_name", "Cruise_line"]:
    indexer = StringIndexer(inputCol=col, outputCol=col+"_index")
    df = indexer.fit(df).transform(df).drop(col)

In [69]:
df.show(3)

+---+------------------+----------+------+------+-----------------+----+---------------+-----------------+
|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Ship_name_index|Cruise_line_index|
+---+------------------+----------+------+------+-----------------+----+---------------+-----------------+
|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|           32.0|             16.0|
|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|           46.0|             16.0|
| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|          134.0|              1.0|
+---+------------------+----------+------+------+-----------------+----+---------------+-----------------+
only showing top 3 rows



In [70]:
for col in ["Ship_name", "Cruise_line"]:
    encoder = OneHotEncoderEstimator(inputCols=[col+"_index"], outputCols=[col+"_encode"])
    df = encoder.fit(df).transform(df).drop(col+"_index")

In [71]:
df.show(3)

+---+------------------+----------+------+------+-----------------+----+-----------------+------------------+
|Age|           Tonnage|passengers|length|cabins|passenger_density|crew| Ship_name_encode|Cruise_line_encode|
+---+------------------+----------+------+------+-----------------+----+-----------------+------------------+
|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55| (137,[32],[1.0])|   (19,[16],[1.0])|
|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55| (137,[46],[1.0])|   (19,[16],[1.0])|
| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|(137,[134],[1.0])|    (19,[1],[1.0])|
+---+------------------+----------+------+------+-----------------+----+-----------------+------------------+
only showing top 3 rows



In [72]:
cols = [x for x in df.columns if x not in ["crew"]]

In [73]:
assembler = VectorAssembler(inputCols=cols, outputCol="features")
df = assembler.transform(df)

In [74]:
df.select("features").head()

Row(features=SparseVector(162, {0: 6.0, 1: 30.277, 2: 6.94, 3: 5.94, 4: 3.55, 5: 42.64, 38: 1.0, 159: 1.0}))

### linear regression

In [75]:
from pyspark.ml.regression import LinearRegression

In [76]:
lr = LinearRegression(featuresCol="features", labelCol="crew")

In [77]:
lr_fit = lr.fit(df)

In [80]:
lr_summary = lr_fit.summary

In [81]:
lr_summary.meanAbsoluteError

0.061237386351426636

In [82]:
lr_summary.r2

0.9979101501710905

In [84]:
lr_summary.residuals.show(3)

+--------------------+
|           residuals|
+--------------------+
|0.002600972317251...|
|0.002600972317251...|
|0.002172918779459465|
+--------------------+
only showing top 3 rows



### evaluation

In [88]:
dtrain, dtest = df.randomSplit(weights=[0.7, 0.3])
print(dtrain.count(), dtest.count())

124 34


In [89]:
lr = LinearRegression(featuresCol="features", labelCol="crew")
lr_fit = lr.fit(dtrain)

In [93]:
lr_sum = lr_fit.summary

In [94]:
lr_eval = lr_fit.evaluate(dtest)

In [95]:
print("train error ", lr_sum.meanAbsoluteError)
print("test error ", lr_eval.meanAbsoluteError)

train error  0.002464109789631894
test error  1.267794174661397


In [96]:
print("RMSE: {}".format(lr_eval.rootMeanSquaredError))
print("MSE: {}".format(lr_eval.meanSquaredError))
print("R2: {}".format(lr_eval.r2))

RMSE: 1.7652894778081794
MSE: 3.1162469404602744
R2: 0.7827880244066334


In [97]:
# R2 of 0.78 is pretty good, let's check the data a little closer
from pyspark.sql.functions import corr

In [98]:
df.select(corr('crew','passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [99]:
df.select(corr('crew','cabins')).show()

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+



### prediction

In [110]:
prediction = lr_fit.transform(dtest)

In [111]:
from pyspark.ml.evaluation import RegressionEvaluator

In [120]:
# metricname : rmse, mse, mae, r2
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="crew", metricName="rmse")

In [121]:
evaluator.evaluate(prediction)

1.7652894778081794

### Optimization

In [122]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.01, 0.5, 2.0])
             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
             .addGrid(lr.maxIter, [1, 5, 10])
             .build())

In [123]:
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

In [125]:
cv_fit = cv.fit(df)